<a href="https://colab.research.google.com/github/rajatmodi62/KaggleSIM/blob/master/%5BIteration2%5D_1_7_SIIM_ACR_KAGGLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!mkdir ~/'.kaggle'

In [0]:
cp '/content/gdrive/My Drive/kaggletoken/kaggle.json' ~/'.kaggle'/

In [4]:
!kaggle datasets download abhishek/siim-png-images

100% 4.49G/4.50G [00:50<00:00, 95.8MB/s]
100% 4.50G/4.50G [00:50<00:00, 96.4MB/s]


In [5]:
!kaggle datasets download jesperdramsch/siim-acr-pneumothorax-segmentation-data

 99% 1.51G/1.52G [00:17<00:00, 53.5MB/s]
100% 1.52G/1.52G [00:17<00:00, 94.3MB/s]


In [15]:
!kaggle competitions download -c siim-acr-pneumothorax-segmentation

  0% 0.00/3.61k [00:00<?, ?B/s]
100% 3.61k/3.61k [00:00<00:00, 3.02MB/s]
  0% 0.00/1.19k [00:00<?, ?B/s]
100% 1.19k/1.19k [00:00<00:00, 1.07MB/s]
  0% 0.00/1.29M [00:00<?, ?B/s]
100% 1.29M/1.29M [00:00<00:00, 86.1MB/s]
  0% 0.00/83.8k [00:00<?, ?B/s]
100% 83.8k/83.8k [00:00<00:00, 73.0MB/s]


In [0]:
!unzip -q  siim-acr-pneumothorax-segmentation-data.zip -d siim-acr-pneumothorax-segmentation-data

In [0]:
!unzip -q siim-png-images.zip -d siim-png-images

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


def dice_loss(input, target):
    """
    input is a torch variable of size BatchxchannelsxHxW representing log probabilities for each class
    target is a 1-hot representation of the groundtruth, shoud have same size as the input
    """
    return 0

In [0]:
#utility functions
#dimensions X_train->[1,1024,1024]
def plot_data(epoch,iteration,X_train,Y_train,Y_pred):

  num_img=3
  figure, axis = plt.subplots(nrows=1, ncols=num_img, sharey=True, figsize=(num_img*10,10))
  axis[0].set_title('Train Example')
  axis[1].set_title('Ground Truth')
  axis[2].set_title('Predicted Sample')
  axis[0].imshow(X_train[0][0].cpu().numpy(),cmap=plt.cm.bone)
  axis[1].imshow(Y_train[0][0].cpu().numpy())
  axis[2].imshow(Y_pred[0][0].cpu().numpy())
  title="Epoch :" +str(epoch)+ "Iteration :"+ str(iteration)
  figure.suptitle(title, fontsize=16)

In [0]:
from torch.utils.data.dataset import Dataset
import torch
import numpy as np
import os
import nibabel as nib
import pandas as pd
from skimage import io, transform
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision.transforms as transform
from PIL import Image, ImageFile
from collections import defaultdict, deque
import collections
from tqdm import tqdm

'''train- img_path, mask_csv_path'''
'''test - img_path, mask_csv_path none'''
class customDataLoader(Dataset):
    
    def rle2mask(self,rle, width, height):
      mask= np.zeros(width* height)
      array = np.asarray([int(x) for x in rle.split()])
      starts = array[0::2]
      lengths = array[1::2]

      current_position = 0
      for index, start in enumerate(starts):
          current_position += start
          mask[current_position:current_position+lengths[index]] = 255
          current_position += lengths[index]

      return mask.reshape(width, height)
  
  
    def __init__(self, img_dir, mask_csv_path,height,width): #READ DATA
       
        
        self.height=height
        self.width=width
        self.img_dir=img_dir
        self.image_info = collections.defaultdict(dict)
        
        if mask_csv_path is not None:
          #train case
          self.mask_csv=pd.read_csv(mask_csv_path)
          #print(self.mask_csv)
          counter = 0
          
          for index, row in tqdm(self.mask_csv.iterrows(), total=len(self.mask_csv)):
              image_id = row['ImageId']
              image_path = os.path.join(img_dir, image_id)
              
              if os.path.exists(image_path + '.png'):
                self.image_info[counter]["image_id"] = image_id
                self.image_info[counter]["image_path"] = image_path

                if row[" EncodedPixels"].strip() != "-1":                  
                  self.image_info[counter]["annotations"] = row[" EncodedPixels"].strip()
                else:
                  self.image_info[counter]["annotations"]="-1" 
                counter += 1
                
            
        
    def __getitem__(self, index): # RETURN ONE ITEM ON THE INDEX
        #read the image
        
        image_path=self.image_info[index]["image_path"]
        img = Image.open(image_path + '.png')
        width,height=img.size
        img=img.resize((self.width,self.height),resample=Image.BILINEAR)
        info=self.image_info[index]
        
        
        if "annotations" in self.image_info[0].keys():
          #check the annotations 
          #image was encoded into its original resoliution
          #decode the mask, treat it as an image , resize it to new resoltion 
          if info['annotations'] !="-1":
            mask= self.rle2mask(info['annotations'],width,height)
            #take a transpose of the mask 
            mask=Image.fromarray(mask.T)
            mask=mask.resize((self.width,self.height),resample=Image.BILINEAR)

          else: 
            mask=np.ones((self.width,self.height))
            mask=Image.fromarray(mask)

          transform_to_tensor=transform.ToTensor()
          img=transform_to_tensor(img)
          mask=transform_to_tensor(mask)

          return (img,mask)
        
          
    def __len__(self): # RETURN THE DATA LENGTH
        return len(self.image_info)

In [0]:
#simple vanilla unet 
class Unet(nn.Module):
  def __init__(self,channels,height,width):
    super(Unet, self).__init__()
    self.nc=channels
    self.h=height
    self.w=width
    self.conv11=nn.Conv2d(self.nc,8,kernel_size=3,padding=1)
    self.conv12=nn.Conv2d(8,8,kernel_size=3,padding=1)
    
    self.conv21=nn.Conv2d(8,16,kernel_size=3,padding=1)
    self.conv22=nn.Conv2d(16,16,kernel_size=3,padding=1)
    
    self.conv31=nn.Conv2d(16,32,kernel_size=3,padding=1)
    self.conv32=nn.Conv2d(32,32,kernel_size=3,padding=1)
    
    self.conv41=nn.Conv2d(32,64,kernel_size=3,padding=1)
    self.conv42=nn.Conv2d(64,64,kernel_size=3,padding=1)
    
    
    self.conv51=nn.Conv2d(64,64,kernel_size=3,padding=1)
    self.conv52=nn.Conv2d(64,64,kernel_size=3,padding=1)
    
    self.conv61=nn.Conv2d(64,128,kernel_size=3,padding=1)
    self.conv62=nn.Conv2d(128,128,kernel_size=3,padding=1)
    self.pool=nn.MaxPool2d(kernel_size=2)
    
    #upsampling network , increate the number of layers, concat, and get the outputs
    self.upconv1=nn.ConvTranspose2d(128,64,kernel_size=2,stride=2)
    
    self.conv71=nn.Conv2d(128,64,kernel_size=3,padding=1)
    self.conv72=nn.Conv2d(64,64,kernel_size=3,padding=1)
    
    self.upconv2=nn.ConvTranspose2d(64,32,kernel_size=2,stride=2)
    
    self.conv81=nn.Conv2d(96,64,kernel_size=3,padding=1)
    self.conv82=nn.Conv2d(64,64,kernel_size=3,padding=1)
    
    self.conv91=nn.Conv2d(64,32,kernel_size=3,padding=1)
    self.conv92=nn.Conv2d(32,32,kernel_size=3,padding=1)
    
    self.upconv3=nn.ConvTranspose2d(32,16,kernel_size=2,stride=2)
    self.conv10_1=nn.Conv2d(32,16,kernel_size=3,padding=1)
    self.conv10_2=nn.Conv2d(16,16,kernel_size=3,padding=1)
    
    self.upconv4=nn.ConvTranspose2d(16,8,kernel_size=2,stride=2)
    self.conv11_1=nn.Conv2d(16,8,kernel_size=3,padding=1)
    self.conv11_2=nn.Conv2d(8,8,kernel_size=3,padding=1)
    
    self.finalconv=nn.Conv2d(8,1,kernel_size=1)

        
  def forward(self,input):
    c1=F.relu(self.conv11(input))
    c1=F.relu(self.conv12(c1))
    p1=self.pool(c1)
    #8X64X64 
    
    c2=F.relu(self.conv21(p1))
    c2=F.relu(self.conv22(c2))
    p2=self.pool(c2)
    #16x32x32
    
    c3=F.relu(self.conv31(p2))
    c3=F.relu(self.conv32(c3))
    p3=self.pool(c3)
    #32X16X16
    
    c4=F.relu(self.conv41(p3))
    c4=F.relu(self.conv42(c4))
    p4=self.pool(c4)
   
    #64X8X8
    c5=F.relu(self.conv51(p4))
    c5=F.relu(self.conv52(c5))
    p5=self.pool(c5)
    
    #64X64X4
    c55=F.relu(self.conv61(p5))
    c55=F.relu(self.conv62(c55))
    
    #128X4X4
    #upsampling network 
    u6=self.upconv1(c55)
    #64X8X8
    u6=torch.cat((u6,c5),1)
    #128X8X8
    c6=F.relu(self.conv71(u6))
    c6=F.relu(self.conv72(c6))
    
    #64X8X8
    u7=self.upconv2(c6)
    u7=torch.cat((u7,c4),1)
    #96X16X16
    c7=F.relu(self.conv81(u7))  
    c7=F.relu(self.conv82(c7))
    #reduce to 64 dimensions
    
    u7=self.upconv2(c7)
    u7=torch.cat((u7,c3),1)
    c7=F.relu(self.conv91(u7))
    c7= F.relu(self.conv92((c7)))
    
    u8=self.upconv3(c7)
    u8=torch.cat((u8,c2),1)
    c8= F.relu(self.conv10_1((u8)))
    c8= F.relu(self.conv10_2((c8)))
    
    u9=self.upconv4(c8)
    u9=torch.cat((u9,c1),1)
    c9= F.relu(self.conv11_1((u9)))
    c9= F.relu(self.conv11_2((c9)))
    ##now need to concat c1
    #print(u9.size())
    
    c10=torch.sigmoid(self.finalconv(c9))
    return c10


In [0]:

import os
import time
import datetime
import numpy as np
import random
import torch
import torch.optim
import torch.utils.data
import torch.backends.cudnn

def train(epoch, num_epochs, net, train_loader):
    mean_val_loss = []
    return mean_val_loss
    # End of train function
    
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        #print("initialized weights")
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        
        

In [6]:
print('::......STARTING SIIM SEGMENTATION ON MMWHS DATASET......::\n\n')

########################################################################################################################
    # DATA LOADING
########################################################################################################################

print('Building Dataset....')
  
# Prepare training dataset for dataloading
train_dataloader_input=customDataLoader('/content/siim-png-images/train_png','/content/siim-acr-pneumothorax-segmentation-data/train-rle.csv',128,128)

 # Training Data loader (Using built in Torch class Dataset, refer to customDataLoader)
train_loader = torch.utils.data.DataLoader(dataset=train_dataloader_input,
                                           batch_size=128,
                                           shuffle=True,
                                           ) #

print('....Dataset built!')

##############################################################s##########################################################
    # SETUP NEURAL NETWORK, LOSS FUNCTION
########################################################################################################################
print('Initializing model...')
num_epochs=500

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
unet= Unet(1,128,128).to(device)


 # Define loss criterion..
  
loss=nn.BCELoss()
optimizer_unet=optim.Adam(unet.parameters())
unet.apply(weights_init)


  6%|▌         | 719/11582 [00:00<00:01, 7186.27it/s]

::......STARTING SIIM SEGMENTATION ON MMWHS DATASET......::


Building Dataset....


100%|██████████| 11582/11582 [00:01<00:00, 7581.78it/s]


....Dataset built!
Initializing model...


Unet(
  (conv11): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv12): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv21): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv22): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv31): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv32): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv41): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv42): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv51): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv52): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv61): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv62): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dila

In [0]:
########################################################################################################################
    # PERFORM NETWORK TRAINING AND SAVE LOSS DATA
########################################################################################################################

print('\n===============================TRAINING BEGINS===============================')

for epoch in range(num_epochs):
  
  running_loss=0
  
  for i,(img,mask) in enumerate(train_loader):
    
    #print(img.size())
    img=img.to(device,dtype=torch.float)
    mask=mask.to(device,dtype=torch.float)
    mask=mask/255
    #print(img.size())
    unet.zero_grad()
    output_mask=unet(img)
    error=loss(output_mask,mask)
    error.backward()
    optimizer_unet.step()
    #output_mask=output_mask>=0.5
#     with torch.no_grad():
#       print("1",np.max(output_mask.cpu().numpy()))
#       print("2",np.mean(mask.cpu().numpy()))
      #print("gradients",list(unet.parameters()))
    running_loss+=error.item()
    if i%10==0:
      print('epoch:[%d], iteration:[%d], loss: %f' %
              (epoch + 1,  i, running_loss / 10))
      running_loss=0
    #remove later
    torch.save({
          'epoch': epoch,
          'model_state_dict': unet.state_dict(),
          'optimizer_state_dict': optimizer_unet.state_dict(),
          'loss': running_loss,
          }, "checkpoint.pth")  
        #plot_data(epoch,i,img,mask,output_mask)
  print('\n===============================SAVING WEIGHTS=============================')   
  torch.save({
          'epoch': epoch,
          'model_state_dict': unet.state_dict(),
          'optimizer_state_dict': optimizer_unet.state_dict(),
          'loss': running_loss,
          }, "checkpoint.pth")
print('\n===============================TRAINING COMPLETE=============================')



===============================TRAINING BEGINS===============================
epoch:[1], iteration:[0], loss: 0.069478
epoch:[1], iteration:[10], loss: 0.689561
epoch:[1], iteration:[20], loss: 0.670219
epoch:[1], iteration:[30], loss: 0.490455
epoch:[1], iteration:[40], loss: 0.241129
epoch:[1], iteration:[50], loss: 0.410600
epoch:[1], iteration:[60], loss: 0.331944


In [101]:
torch.cuda.max_memory_allocated(device=None)

1832665088

In [27]:
print('====================TESTING SIIM SEGMENTATION====================')
print('Initializing Neural Network........................')


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# change processing device to gpu
unet= Unet(1,128,128).to(device)
unet.eval()



# Load old weights
checkpoint = torch.load('checkpoint.pth')
unet.load_state_dict(checkpoint['model_state_dict'])

print('....dummyUnet Network loaded with weights')


# Build dataset for loading into network!!
print('Building testing dataset......')
  
test_data_path = '/content/siim-png-images/test_png'

# Prepare validation dataset for dataloading


submission_csv=pd.read_csv('sample_submission.csv')

#drop duplicates from the submission_csv
masks_ = submission_csv.groupby('ImageId')['ImageId'].count().reset_index(name='N')
masks_ = masks_.loc[masks_.N > 1].ImageId.values
###
submission_csv = submission_csv.drop_duplicates('ImageId', keep='last').reset_index(drop=True)



====================TESTING SIIM SEGMENTATION====================
Initializing Neural Network........................
....dummyUnet Network loaded with weights
Building testing dataset......


In [0]:


# Testing Data loader (Using built in Torch class Dataset, refer to dataset.py)
test_loader = torch.utils.data.DataLoader(dataset=test_dataloader_input,
                                          batch_size=1,
                                          shuffle=True,
                                          pin_memory=True,
                                          num_workers=4)  
print('...............Dataset Built!')
print('===============================TESTING BEGINS===============================')

mkdir('result')
copyfile('../input/siim-acr-pneumothorax-segmentation/sample_submission.csv', './result/sample_result.csv')

print('===============================TESTING COMPLETE===============================')
print('segmentation result is saved into ./result/sample_result.csv ')




In [29]:
submission_csv.head()

,ImageId,EncodedPixels
0,1.2.276.0.7230010.3.1.4.8323329.6567.151787519...,-1
1,1.2.276.0.7230010.3.1.4.8323329.6170.151787519...,-1
2,1.2.276.0.7230010.3.1.4.8323329.6346.151787519...,-1
3,1.2.276.0.7230010.3.1.4.8323329.6890.151787520...,-1
4,1.2.276.0.7230010.3.1.4.8323329.6400.151787519...,-1


In [136]:
def mask2rle(img, width, height):
  rle = []
  lastColor = 0;
  currentPixel = 0;
  runStart = -1;
  runLength = 0;

  for x in range(width):
      for y in range(height):
          currentColor = img[x][y]
          if currentColor != lastColor:
              if currentColor == 255:
                  runStart = currentPixel;
                  runLength = 1;
              else:
                  rle.append(str(runStart));
                  rle.append(str(runLength));
                  runStart = -1;
                  runLength = 0;
                  currentPixel = 0;
          elif runStart > -1:
              runLength += 1
          lastColor = currentColor;
          currentPixel+=1;

  return " ".join(rle)

transform_to_tensor=transform.ToTensor()

test_image_height=128
test_image_width=128
threshold=127
sublist=[]

for index, row in tqdm(submission_csv.iterrows(),total=len(submission_csv)):
  image_id=row['ImageId']
  if image_id in list(masks_.ImageId):
    img_path = os.path.join('/content/siim-png-images/test_png', image_id + '.png')
    img = Image.open(img_path)
    width, height = img.size
    img = img.resize((test_image_height, test_image_width), resample=Image.BILINEAR)
    img=transform_to_tensor(img)
    img=img.unsqueeze_(0)
    result = unet(img.to(device))
    #transform the result to pil image, resize to width and height,
    result=transform.ToPILImage()(result.cpu().detach()[0])
    result = np.asarray(result.resize((width, height), resample=Image.BILINEAR))
    result=(result*255>threshold).astype(np.uint8).T
    #print(result)
    #if any ones is there, then segmentaion found 
    
    if np.max(result)==1:
      rle=mask2rle(result,width,height)
      print(rle)
      sublist.append([image_id,rle])
    else:
      rle='-1'
      sublist.append([image_id,rle])

#create a submission_csv
submission_df = pd.DataFrame(sublist, columns=submission_csv.columns.values)
submission_df.to_csv("submission.csv", index=False)












 80%|███████▉  | 1093/1372 [00:27<00:07, 38.81it/s]










 80%|███████▉  | 1097/1372 [00:27<00:07, 39.15it/s]










 80%|████████  | 1101/1372 [00:27<00:06, 39.26it/s]










 81%|████████  | 1105/1372 [00:27<00:06, 39.27it/s]










 81%|████████  | 1110/1372 [00:27<00:06, 39.85it/s]










 81%|████████  | 1114/1372 [00:27<00:06, 39.34it/s]










 81%|████████▏ | 1118/1372 [00:27<00:06, 39.19it/s]










 82%|████████▏ | 1122/1372 [00:27<00:06, 39.35it/s]










 82%|████████▏ | 1127/1372 [00:27<00:06, 39.95it/s]










 83%|████████▎ | 1132/1372 [00:28<00:05, 40.10it/s]










 83%|████████▎ | 1137/1372 [00:28<00:05, 39.77it/s]










 83%|████████▎ | 1142/1372 [00:28<00:05, 40.10it/s]










 84%|████████▎ | 1147/1372 [00:28<00:05, 39.91it/s]










 84%|████████▍ | 1152/1372 [00:28<00:05, 40.38it/s]










 84%|████████▍ | 1157/1372 [00:28<00:05, 40.51it/s]










 85%|████████▍ | 1162/1372 [00:28<00:05, 40.38it/s]









In [142]:
!kaggle competitions submit -f submission.csv -m "my submission" siim-acr-pneumothorax-segmentation 

100% 77.5k/77.5k [00:01<00:00, 41.2kB/s]
Successfully submitted to SIIM-ACR Pneumothorax Segmentation

In [140]:
!kaggle competitions list

ref                                                deadline             category            reward  teamCount  userHasEntered  
-------------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
digit-recognizer                                   2030-01-01 00:00:00  Getting Started  Knowledge       3118            True  
titanic                                            2030-01-01 00:00:00  Getting Started  Knowledge      11527            True  
house-prices-advanced-regression-techniques        2030-01-01 00:00:00  Getting Started  Knowledge       4495            True  
imagenet-object-localization-challenge             2029-12-31 07:00:00  Research         Knowledge         46            True  
competitive-data-science-predict-future-sales      2019-12-31 23:59:00  Playground           Kudos       3595            True  
open-images-2019-object-detection                  2019-10-01 23:59:00  Research           $25,000      

In [141]:
!kaggle  competitions submit siim-acr-pneumothorax-segmentation 

usage: kaggle competitions [-h]
                           {list,files,download,submit,submissions,leaderboard}
                           ...
kaggle competitions: error: too few arguments
